In [1]:
#importing necessary modules
import pandas as pd
import numpy as np
import time
import seaborn as sns
from matplotlib import pyplot as plt
import sqlite3
%matplotlib inline

In [11]:
products = pd.read_csv('ia_products.csv')
product_price = pd.read_csv('ia_product_price.csv')
vendors = pd.read_csv('ia_vendors.csv')
stores = pd.read_csv('ia_stores.csv')
transactions = pd.read_csv('transactions.csv')

In [3]:
products.columns

Index(['Unnamed: 0', 'ItemNumber', 'Category', 'CategoryName',
       'ItemDescription', 'BottleVolume(ml)', 'Proof', 'Imported', 'Type'],
      dtype='object')

In [14]:
products.drop(['Unnamed: 0'], axis=1, inplace=True)

In [15]:
conn = sqlite3.connect('liquor_rdb.db')
#make cursor
cursor = conn.cursor()
#make products table
sql_query = '''CREATE TABLE products(
                   'ItemNumber' varchar(50) PRIMARY KEY,
                   'Category' float(64),
                   'CategoryName' varchar(50),
                   'ItemDescription' varchar(50),
                   'Bottlevolume(ml)' float(64),
                   'Proof' float(64),
                   'Imported' varchar(50),
                   'Type' varchar(50));'''

cursor.execute(sql_query)
# insert data
products.to_sql('products',conn,if_exists='append',index=False)

cursor.close()
conn.close()

In [5]:
product_price.columns

Index(['ItemNumber', 'Date', 'StateBottleRetail', 'StateBottleCost',
       'StateBottleProfit'],
      dtype='object')

In [16]:
conn = sqlite3.connect('liquor_rdb.db')
#make cursor
cursor = conn.cursor()
#make product-price table
sql_query = '''CREATE TABLE product_price(
                   'ItemNumber' varchar(50),
                   'Date' datetime64[ns],
                   'StateBottleRetail' float(64),
                   'StateBottleCost' float(64),
                   'StateBottleProfit' float(64),
                   primary key ('ItemNumber', 'Date'),
                   FOREIGN KEY ('ItemNumber') REFERENCES products('ItemNumber')
                   );'''
cursor.execute(sql_query)
#insert data
product_price.to_sql('product_price',conn,if_exists='append',index=False)

cursor.close()
conn.close()

In [6]:
stores.columns

Index(['Unnamed: 0', 'StoreNumber', 'StoreName', 'County', 'CountyNumber',
       'City', 'ZipCode', 'StoreLocation', 'Address', 'StoreType'],
      dtype='object')

In [13]:
stores.drop(['Unnamed: 0'], axis=1, inplace=True)

In [17]:
conn = sqlite3.connect('liquor_rdb.db')
#make cursor
cursor = conn.cursor()
#make stores table
sql_query = '''CREATE TABLE stores(
                   'StoreNumber' INTEGER PRIMARY KEY,
                   'StoreName' varchar(250),
                   'County' varchar(250),
                   'CountyNumber' varchar(250),
                   'City' varchar(250),
                   'ZipCode' varchar(250),
                   'StoreLocation' varchar(250),
                   'Address' varchar(250),
                   'StoreType' varchar(250));'''

cursor.execute(sql_query)
#insert data
stores.to_sql('stores',conn,if_exists='append',index=False)

cursor.close()
conn.close()

In [9]:
transactions.columns

Index(['InvoiceID', 'Date', 'StoreNumber', 'Category', 'VendorNumber',
       'ItemNumber', 'BottlesSold', 'Sale(USD)', 'VolumeSold(Liters)'],
      dtype='object')

In [18]:
conn = sqlite3.connect('liquor_rdb.db')
#make cursor
cursor = conn.cursor()
#make transactions table
sql_query = '''CREATE TABLE transactions(
                   'InvoiceID' varchar(50) PRIMARY KEY,
                   'Date' datetime64[ns],
                   'StoreNumber' varchar(50),
                   'Category' varchar(50),
                   'VendorNumber' varchar(50),
                   'ItemNumber' varchar(50),
                   'BottlesSold' interger,
                   'Sale(USD)' float(64),
                   'VolumeSold(Liters)' float(64),
                   FOREIGN KEY ('VendorNumber') REFERENCES vendors('VendorNumber'),
                   FOREIGN KEY ('ItemNumber','Date') REFERENCES product_price('ItemNumber','Date'),
                   FOREIGN KEY ('StoreNumber') REFERENCES stores('StoreNumber')
                   );'''
cursor.execute(sql_query)
# insert data
transcations.to_sql('transactions',conn,if_exists='append',index=False)

cursor.close()
conn.close()

In [12]:
vendors.columns

Index(['Unnamed: 0', 'VendorNumber', 'VendorName'], dtype='object')

In [19]:
vendors.drop(['Unnamed: 0'], axis=1, inplace=True)

In [20]:
conn = sqlite3.connect('liquor_rdb.db')
#make cursor
cursor = conn.cursor()
#make vendor table
sql_query = '''CREATE TABLE vendors(
                   'VendorNumber' varchar(50) PRIMARY KEY,
                   'VendorName' varchar(50));'''
cursor.execute(sql_query)
# insert data
vendors.to_sql('vendors',conn,if_exists='append',index=False)

cursor.close()
conn.close()

In [21]:
#fetching table 
conn = sqlite3.connect('liquor_rdb.db')
cursor = conn.cursor()
#select data and transform to df
query = '''SELECT * FROM products  ;'''
cursor.execute(query)
result = cursor.fetchall()
cursor.close()
conn.close()

In [22]:
len(result)

11728

In [23]:

pd.DataFrame(result,columns=['ItemNumber', 'Category', 'CategoryName', 'ItemDescription',
       'BottleVolume(ml)', 'Proof', 'Imported', 'Type'])

,ItemNumber,Category,CategoryName,ItemDescription,BottleVolume(ml),Proof,Imported,Type
0,26826,1011400.0,tennessee whiskies,jack daniels old #7 black lbl,750.0,80.0,domestic,whiskey
1,69706,1082200.0,imported schnapps,dr. mcgillicuddy's root beer schnapps,750.0,42.0,imported,schnapps
2,37664,1031100.0,american vodkas,row vodka mini,0.0,80.0,domestic,vodka
3,37258,1031100.0,american vodkas,missing,1000.0,80.0,domestic,vodka
4,43026,1062400.0,spiced rum,admiral nelson spiced rum,750.0,70.0,domestic,rum
...,...,...,...,...,...,...,...,...
11723,977249,1091100.0,american distilled spirit specialty,mark and diggers rye apple moonshine mini,50.0,80.0,domestic,whiskey
11724,977237,1091100.0,american distilled spirit specialty,mark and diggers hazelnut rum mini,50.0,80.0,domestic,rum
11725,977241,1091100.0,american distilled spirit specialty,appalachian apple pie moonshine mini,50.0,50.0,domestic,whiskey
11726,978152,1091100.0,american distilled spirit specialty,ryder cup lemonade moonshine mini,50.0,40.0,domestic,whiskey
